In [16]:
# Gerekli kütüphaneleri import edelim
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RandomizedSearchCV, KFold
from catboost import CatBoostRegressor
import warnings

warnings.filterwarnings('ignore')

# 1. Veri Setini Yükleme
print("1. Veri Setini Yükleme")
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_x.csv')

# Kontrollü sütun adları kontrolü
if 'Degerlendirme Puani' not in train_df.columns:
    print("Degerlendirme Puani sütunu bulunamadı. Lütfen sütun adlarını kontrol edin.")
    print(train_df.columns)
else:
    print("'Degerlendirme Puani' sütunu başarıyla yüklendi.")

# Hedef değişkende eksik değerleri kontrol edip silme
train_df.dropna(subset=['Degerlendirme Puani'], inplace=True)

# 2. Veri Ön İşleme ve Temizleme

# Yaş hesaplama fonksiyonu
def calculate_age(born, application_year):
    try:
        born = pd.to_datetime(born, errors='coerce', infer_datetime_format=True)
        age = application_year - born.year
        return int(age)
    except:
        return np.nan

# Yaşları hesaplayalım
train_df['Yas'] = train_df.apply(lambda row: calculate_age(row['Dogum Tarihi'], row['Basvuru Yili']), axis=1)
test_df['Yas'] = test_df.apply(lambda row: calculate_age(row['Dogum Tarihi'], row['Basvuru Yili']), axis=1)

# Üniversite okuma sayısı
def calculate_university_count(row):
    if pd.isna(row['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']):
        return 1
    else:
        return 2

train_df['Universite Okuma Sayisi'] = train_df.apply(calculate_university_count, axis=1)
test_df['Universite Okuma Sayisi'] = test_df.apply(calculate_university_count, axis=1)

# Gereksiz sütunları kaldırma
columns_to_drop = ['Dogum Tarihi', 'Lise Adi', 'Lise Adi Diger', 'Lise Bolum Diger', 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

# Binary sütunların kodlanması
binary_columns = [
    'Burs Aliyor mu?', 'Daha Once Baska Bir Universiteden Mezun Olmus',
    'Baska Bir Kurumdan Burs Aliyor mu?', 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
    'Profesyonel Bir Spor Daliyla Mesgul musunuz?', 'Aktif olarak bir STK üyesi misiniz?',
    'Stk Projesine Katildiniz Mi?', 'Girisimcilikle Ilgili Deneyiminiz Var Mi?', 'Ingilizce Biliyor musunuz?'
]

for col in binary_columns:
    for df in [train_df, test_df]:
        df[col] = df[col].astype(str).str.strip().str.lower().fillna('bilinmiyor').map({'evet': 1, 'hayır': 0})

# Cinsiyet ve Üniversite Türü sütunlarının kodlanması
for df in [train_df, test_df]:
    df['Cinsiyet'] = df['Cinsiyet'].astype(str).str.strip().str.lower().fillna('bilinmiyor').map({'erkek': 2, 'kadın': 1, 'belirtmek istemiyorum': 0})
    df['Universite Turu'] = df['Universite Turu'].astype(str).str.strip().str.lower().fillna('bilinmiyor').map({'özel': 1, 'devlet': 0})

# Eksik değerleri doldurma
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()
numerical_columns = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Hedef sütunun olup olmadığını kontrol ederek devam et
if 'Degerlendirme Puani' in numerical_columns:
    numerical_columns.remove('Degerlendirme Puani')  # Hedef değişkeni çıkardık

for df in [train_df, test_df]:
    for col in numerical_columns:
        median = train_df[col].median()
        df[col].fillna(median, inplace=True)
    for col in categorical_columns:
        df[col] = df[col].astype(str).fillna('Bilinmiyor')

# 3. Özellik Mühendisliği - Anne ve Baba Eğitim Seviyesi
education_mapping = {
    'Eğitim Yok': 0,
    'İlkokul Mezunu': 1,
    'Ortaokul Mezunu': 2,
    'Lise': 3,
    'Üniversite': 4,
    'Yüksek Lisans / Doktora': 5,
    'bilinmiyor': 0,
    'Bilinmiyor': 0
}

for df in [train_df, test_df]:
    df['Anne Egitim Seviyesi'] = df['Anne Egitim Durumu'].astype(str).str.strip().map(education_mapping).fillna(0)
    df['Baba Egitim Seviyesi'] = df['Baba Egitim Durumu'].astype(str).str.strip().map(education_mapping).fillna(0)
    df['Ebeveyn Egitim Ort'] = (df['Anne Egitim Seviyesi'] + df['Baba Egitim Seviyesi']) / 2

# 4. Sosyal Katılım ve Gelir Skorları
print(" - Sosyal Katılım ve Gelir Skorları oluşturuluyor")
for df in [train_df, test_df]:
    df['Sosyal Katılım Skoru'] = df['Aktif olarak bir STK üyesi misiniz?'] + df['Stk Projesine Katildiniz Mi?'] + df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?']
    df['Sosyo-Ekonomik Skor'] = df['Anne Egitim Seviyesi'] + df['Baba Egitim Seviyesi'] + df.get('Aile Geliri', 0)  # Eğer 'Aile Geliri' sütunu yoksa varsayılanı 0 alıyoruz

# Zaman Serisi Trend Analizi
print(" - Zaman Serisi ile ilgili özellikler oluşturuluyor")

# Başvuru Yılı Trendi
train_df['Degerlendirme Puani Rolling Mean'] = train_df.groupby('Basvuru Yili')['Degerlendirme Puani'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
train_df['Degerlendirme Puani Yillik Degisim'] = train_df.groupby('Basvuru Yili')['Degerlendirme Puani'].transform(lambda x: x.pct_change().fillna(0))

# Test verisi için başvuru yılı ortalamalarını kullan
mean_puan_by_year = train_df.groupby('Basvuru Yili')['Degerlendirme Puani'].mean().to_dict()
test_df['Degerlendirme Puani Rolling Mean'] = test_df['Basvuru Yili'].map(mean_puan_by_year)
test_df['Degerlendirme Puani Yillik Degisim'] = test_df['Basvuru Yili'].map(lambda x: mean_puan_by_year.get(x, 0) - mean_puan_by_year.get(x-1, 0))

# Modelleme
print("4. Modelleme")

# Hedef değişkeni ve özellikleri ayırma
X = train_df.drop(['Degerlendirme Puani', 'id', 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'], axis=1)
y = train_df['Degerlendirme Puani']

test_features = test_df.drop(['id', 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'], axis=1)

# CatBoost ile modelleme
print(" - CatBoost modeli için eğitim başlıyor")
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# CatBoost modeli ve hiperparametre optimizasyonu
model = CatBoostRegressor(
    verbose=0, random_state=42, loss_function='RMSE'
)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
randomized_search = RandomizedSearchCV(
    estimator=model,
    param_distributions={
        'iterations': [500, 1000],
        'depth': [4, 6, 8],
        'learning_rate': [0.01, 0.03, 0.1],
        'l2_leaf_reg': [1, 3, 5]
    },
    cv=kf,
    scoring='neg_mean_squared_error',
    n_iter=20,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

randomized_search.fit(X, y, cat_features=categorical_cols)

best_rmse = np.sqrt(-randomized_search.best_score_)
print(f"En iyi RMSE: {best_rmse}")
print(f"En iyi hiperparametreler: {randomized_search.best_params_}")

# Model ile tahmin
best_model = randomized_search.best_estimator_
best_model.fit(X, y, cat_features=categorical_cols)

# Test verisi üzerinde tahmin yapalım
print(" - Test verisi üzerinde tahmin yapılıyor")
test_predictions = best_model.predict(test_features)

# Negatif tahminleri sıfıra eşitle
test_predictions = np.where(test_predictions < 0, 0, test_predictions)

# Sonuçları kaydet
submission = pd.DataFrame({
    'id': test_df['id'],
    'Degerlendirme Puani': test_predictions
})
submission.to_csv('submission.csv', index=False)
print("\nTahminler 'submission.csv' dosyasına kaydedildi.")


1. Veri Setini Yükleme
'Degerlendirme Puani' sütunu başarıyla yüklendi.
 - Sosyal Katılım ve Gelir Skorları oluşturuluyor
 - Zaman Serisi ile ilgili özellikler oluşturuluyor
4. Modelleme
 - CatBoost modeli için eğitim başlıyor
Fitting 5 folds for each of 20 candidates, totalling 100 fits
En iyi RMSE: 3.2925630602737685
En iyi hiperparametreler: {'learning_rate': 0.1, 'l2_leaf_reg': 3, 'iterations': 1000, 'depth': 6}
 - Test verisi üzerinde tahmin yapılıyor

Tahminler 'submission.csv' dosyasına kaydedildi.
